In [ ]:
from bs4 import BeautifulSoup
import urllib
import json   
import requests
import re
from datetime import date

In [ ]:
# сегодняшняя дата
today = str(date.today())

# вводим необходимые заголовки (ПКМ - посмотреть код - Network)
HEADERS = {
    'Accept': '###',
    'User-Agent': '###'
}


def get_html(url, params=''):
    r = requests.get(url, headers=HEADERS, params=params)
    return r


def extract_simple_table(soup):  # простые таблицы ("Рукоположение / Постриг / Возведение в сан", "Награды", "Преследования")
    values = []
    position = soup.find("table").find_all("tr")
    for item in position:
        info = dict()
        info["Дата"] = item.find('td', class_="date").get_text(strip=True)
        info["Описание"] = item.find('td', class_="details").get_text(strip=True)
        values.append(info)
    return values


def extract_table_with_head(soup):  # таблицы с заголовками ("Образование" и "Места служения / Должности")
    values = []
    keys = soup.find('thead').find_all('th')
    lines = soup.find('tbody').find_all('tr')
    for line in range(0, len(lines)):
        cells = lines[line].find_all('td')
        info = dict()
        for cell in range(0, len(cells)):
            info[keys[cell].get_text(strip=True)] = cells[cell].get_text(strip=True)
        values.append(info)
    return values


def extract_relations(soup):  # таблица "Родственники"
    values = dict()
    items = soup.find_all('li')
    for item in items:
        if(item.find('strong') is None):
            continue
        relation = item.find('strong').get_text(strip=True)
        name = item.get_text(strip=True).split(relation)[1].strip('—').strip(' ')  # разделение по "—", так как часто бывает нарушение структуры html
        values[relation] = name
    return values


def extract_text(soup):  # "Другие сведения"
    return soup.find(class_="person__text").get_text(strip=True)


def extract_list(soup):  # списки ("Архивные источники", "Литература", "Сочинения")
    values = []
    for item in soup.find_all("li"):
        values.append(item.get_text(strip=True))
    return values


def parse_person_block(soup):
    key = soup.find("h2").get_text(strip=True)
    values = []
    if(key == "Родственники"):
        values = extract_relations(soup)
    elif(key == "Рукоположение / Постриг / Возведение в сан"):
        values = extract_simple_table(soup)
    elif(key == "Награды"):
        values = extract_simple_table(soup)
    elif(key == "Преследования"):
        values = extract_simple_table(soup)
    elif(soup.find("thead") is not None):
        values = extract_table_with_head(soup)
    elif(soup.find(class_="person__list") is not None):
        values = extract_list(soup)
    elif(soup.find(class_="person__text") is not None):
        values = extract_text(soup)
    return (key, values)


def parse_all_person_blocks(soup, values):
    for block in soup.find_all("div", class_="person__block"):
        if(block.find("h2") is None):
            continue
        (key, value) = parse_person_block(block)
        values[key] = value


def parse_person_details(soup, values):
    for tr in soup.find(class_='person-detail__info').find_all("tr"):
        ths = tr.find_all("th")
        tds = tr.find_all("td")
        for i in range(0, len(ths)):
            values[ths[i].get_text(strip=True)] = tds[i].get_text(strip=True)


def parse_proslavlenie(soup, values):
    if(soup.find(class_="col-1-1") is not None):
        values['Прославление'] = soup.find(class_="col-1-1").get_text(strip=True)


def extract_values(soup, url):
    values = dict()
    values["original_link"] = url
    values["retrieved_at"] = today
    values['ФИО'] = soup.find('div', class_='person-detail__info').find('h1').get_text(strip=True)
    parse_proslavlenie(soup, values)
    parse_person_details(soup, values)
    parse_all_person_blocks(soup, values)
    return values


def get_content(html, url):
    soup = BeautifulSoup(html, 'html.parser')
    return extract_values(soup, url)


def parse_person(n):
    url = 'https://www.pravoslavnoe-duhovenstvo.ru/person/' + str(n)
    return get_content(get_html(url).text, url)

# начинаем скачивать данные
parsed = list()

# указываем в range, какой диапазон страниц мы хотим распарсить
# например, range(0, 12000):
for i in range(0, 0):
    try:  # напечатает, получилось ли извлечь страницу
        print(i)
        parsed.append(parse_person(i))
    except Exception as e:
        print(str(e))

# называем json и записываем туда данные
with open('test.json', 'w', encoding='utf-8') as f:
        json.dump(parsed, f, ensure_ascii=False, indent=4)


In [ ]:
# приводим данные в порядок

# открываем скачанные данные
with open('test.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


# в "православном духовенстве" в поле "ФИО" указывают также сан или должность человека через запятую
# например "Хромцов Матвей Иванович, священник"
# чтобы получить отдельно ФИО, отдельно должность, разделяем поле по запятой
for d in data:
    FIO = d['ФИО'].split(', ')
    d['ФИО'] = FIO[0]
    if (len(FIO) == 1):
        continue
    else:
        d['Должность'] = FIO[1]

# замена названий ключей на новые единые названия из таблицы унифицированных полей
for d in data:
    try:
        d['fullname'] = d['ФИО']
        del d['ФИО']
        d['ecclesiastical_title'] = d['Должность']
        del d['Должность']
    except Exception as e:
        del e

for d in data:
    try:
        d['glorification'] = d['Прославление']
        del d['Прославление']
    except Exception as e:
        del e

for d in data:
    try:
        d['birthdate'] = d['Дата рождения:']
        del d['Дата рождения:']
    except Exception as e:
        del e

for d in data:
    try:
        d['birthplace'] = d['Место рождения:']
        del d['Место рождения:']
    except Exception as e:
        del e

for d in data:
    try:
        d['death_date'] = d['Дата смерти:']
        del d['Дата смерти:']
    except Exception as e:
        del e

for d in data:
    try:
        d['death_place'] = d['Место смерти:']
        del d['Место смерти:']
    except Exception as e:
        del e

for d in data:
    try:
        ed = d['Образование']
        for i in ed:
            try:
                i['education_start_year'] = i['Дата поступления']
                del i['Дата поступления']
            except Exception as e:
                del e
            try:
                i['education_end_year'] = i['Дата окончания']
                del i['Дата окончания']
            except Exception as e:
                del e
            try:
                i['seminary'] = i['Учебное заведение']
                del i['Учебное заведение']
            except Exception as e:
                del e
            try:
                i['education_status'] = i['Комментарий']
                del i['Комментарий']
            except Exception as e:
                del e
    except Exception as e:
        del e

for d in data:
    try:
        rep = d['Преследования']
        for i in rep:
            try:
                i['repression_date'] = i['Дата']
                del i['Дата']
            except Exception as e:
                del e
            try:
                i['repression_info'] = i['Описание']
                del i['Описание']
            except Exception as e:
                del e
    except Exception as e:
        del e

for d in data:
    try:
        ordination = d['Рукоположение / Постриг / Возведение в сан']
        for i in ordination:
            try:
                i['ordination_date'] = i['Дата']
                del i['Дата']
            except Exception as e:
                del e
            try:
                i['ordination_status'] = i['Описание']
                del i['Описание']
            except Exception as e:
                del e
    except Exception as e:
        del e

for d in data:
    try:
        ordination = d['Места служения / Должности']
        for i in ordination:
            try:
                i['job_start_date'] = i['Дата начала']
                del i['Дата начала']
            except Exception as e:
                del e
            try:
                i['job_end_date'] = i['Дата окончания']
                del i['Дата окончания']
            except Exception as e:
                del e
            try:
                i['job_info'] = i['Место служения, сан, должность']
                del i['Место служения, сан, должность']
            except Exception as e:
                del e
    except Exception as e:
        del e

for d in data:
    try:
        awards = d['Награды']
        for i in awards:
            try:
                i['award_date'] = i['Дата']
                del i['Дата']
            except Exception as e:
                del e
            try:
                i['award'] = i['Описание']
                del i['Описание']
            except Exception as e:
                del e
    except Exception as e:
        del e

for d in data:
    if('Родственники' in d):
        rel = d['Родственники']
        if('отец' in rel):
            rel['father'] = rel['отец']
            del rel['отец']
        if ('мать' in rel):
            rel['mother'] = rel['мать']
            del rel['мать']
        if ('муж' in rel):
            rel['husband'] = rel['муж']
            del rel['муж']
        if ('жена' in rel):
            rel['wife'] = rel['жена']
            del rel['жена']
        if ('дочь' in rel):
            rel['daughter'] = rel['дочь']
            del rel['дочь']
        if ('сын' in rel):
            rel['son'] = rel['сын']
            del rel['сын']
        if ('брат' in rel):
            rel['brother'] = rel['брат']
            del rel['брат']
        if ('сестра' in rel):
            rel['sister'] = rel['сестра']
            del rel['сестра']
        if ('бабушка' in rel):
            rel['grandmother'] = rel['бабушка']
            del rel['бабушка']
        if ('дед' in rel):
            rel['grandfather'] = rel['дед']
            del rel['дед']
        if ('внучка' in rel):
            rel['granddaughter'] = rel['внучка']
            del rel['внучка']
        if ('внук' in rel):
            rel['grandson'] = rel['внук']
            del rel['внук']
        if ('зять' in rel):
            rel['son_in_law'] = rel['зять']
            del rel['зять']
        if ('тетя' in rel):
            rel['aunt'] = rel['тетя']
            del rel['тетя']
        if ('дядя' in rel):
            rel['uncle'] = rel['дядя']
            del rel['дядя']
        if ('другие родственники' in rel):
            rel['other_relatives'] = rel['другие родственники']
            del rel['другие родственники']
        if ('другие члены семьи' in rel):
            rel['other_relatives'] = rel['другие члены семьи']
            del rel['другие члены семьи']
        if ('тесть' in rel):
            rel['father_in_law'] = rel['тесть']
            del rel['тесть']
        if ('теща' in rel):
            rel['mother_in_law'] = rel['теща']
            del rel['теща']
        if ('прадед' in rel):
            rel['great_grandfather'] = rel['прадед']
            del rel['прадед']
        if ('правнук' in rel):
            rel['great_grandson'] = rel['правнук']
            del rel['правнук']
        if ('праправнук' in rel):
            rel['great_great_grandson'] = rel['праправнук']
            del rel['праправнук']
        if ('другие дети' in rel):
            rel['children'] = rel['другие дети']
            del rel['другие дети']
        if ('дети' in rel):
            rel['children'] = rel['дети']
            del rel['дети']
        if ('дядя (брат отца)' in rel):
            rel['uncle_fathers_brother'] = rel['дядя (брат отца)']
            del rel['дядя (брат отца)']
        if ('дядя (брат матери)' in rel):
            rel['uncle_mothers_brother'] = rel['дядя (брат матери)']
            del rel['дядя (брат матери)']
        if ('тетя (сестра матери)' in rel):
            rel['aunt_mothers_sister'] = rel['тетя (сестра матери)']
            del rel['тетя (сестра матери)']
        if ('тетя (сестра отца)' in rel):
            rel['aunt_fathers_sister'] = rel['тетя (сестра отца)']
            del rel['тетя (сестра отца)']
        if('племянник (сын брата)' in rel):
            rel['nephew_brothers_son'] = rel['племянник (сын брата)']
            del rel['племянник (сын брата)']
        if ('племянница (дочь брата)' in rel):
            rel['niece_brothers_daughter'] = rel['племянница (дочь брата)']
            del rel['племянница (дочь брата)']
        if ('сестра жены' in rel):
            rel['wifes_sister'] = rel['сестра жены']
            del rel['сестра жены']
        if ('брат жены' in rel):
            rel['wifes_brother'] = rel['брат жены']
            del rel['брат жены']
        if ('муж сестры жены' in rel):
            rel['wifes_sisters_husband'] = rel['муж сестры жены']
            del rel['муж сестры жены']
        if ('муж сестры' in rel):
            rel['sisters_husband'] = rel['муж сестры']
            del rel['муж сестры']
        if ('отец невестки' in rel):
            rel['father_of_daughter_in_law'] = rel['отец невестки']
            del rel['отец невестки']
        if ('дед жены' in rel):
            rel['wifes_grandfather'] = rel['дед жены']
            del rel['дед жены']
        if ('муж внучки' in rel):
            rel['granddaughters_husband'] = rel['муж внучки']
            del rel['муж внучки']
        if ('невестка (жена сына)' in rel):
            rel['daughter_in_law'] = rel['невестка (жена сына)']
            del rel['невестка (жена сына)']
        if('родственник (точная степень родства не установлена)' in rel):
            rel['other_relatives'] = rel['родственник (точная степень родства не установлена)']
            del rel['родственник (точная степень родства не установлена)']
        if ('праправнучка' in rel):
            rel['great_great_granddaughter'] = rel['праправнучка']
            del rel['праправнучка']
        if ('племянница (дочь сестры)' in rel):
            rel['niece_sisters_daughter'] = rel['племянница (дочь сестры)']
            del rel['племянница (дочь сестры)']
        if ('племянник (сын сестры)' in rel):
            rel['nephew_sisters_son'] = rel['племянник (сын сестры)']
            del rel['племянник (сын сестры)']
        if ('жена брата' in rel):
            rel['brothers_wife'] = rel['жена брата']
            del rel['жена брата']
        if ('двоюродный брат' in rel):
            rel['cousin'] = rel['двоюродный брат']
            del rel['двоюродный брат']
        if ('двоюродная сестра' in rel):
            rel['cousin'] = rel['двоюродная сестра']
            del rel['двоюродная сестра']
        if ('отец зятя' in rel):
            rel['father_of_son_in_law'] = rel['отец зятя']
            del rel['отец зятя']
        if ('мачеха' in rel):
            rel['stepmother'] = rel['мачеха']
            del rel['мачеха']
        if ('правнучка' in rel):
            rel['great_granddaughter'] = rel['правнучка']
            del rel['правнучка']

for d in data:
    if ('Родственники' in d):
        d['relatives'] = d['Родственники']
        del d['Родственники']
    if ('Образование' in d):
        d['education'] = d['Образование']
        del d['Образование']
    if ('Рукоположение / Постриг / Возведение в сан' in d):
        d['ordination'] = d['Рукоположение / Постриг / Возведение в сан']
        del d['Рукоположение / Постриг / Возведение в сан']
    if ('Места служения / Должности' in d):
        d['job'] = d['Места служения / Должности']
        del d['Места служения / Должности']
    if ('Награды' in d):
        d['awards'] = d['Награды']
        del d['Награды']
    if ('Преследования' in d):
        d['repression'] = d['Преследования']
        del d['Преследования']
    if ('Иконы' in d):
        d['icons'] = d['Иконы']
        del d['Иконы']
    if ('Фотографии' in d):
        d['photo'] = d['Фотографии']
        del d['Фотографии']
    if ('Постриг' in d):
        d['tonsure'] = d['Постриг']
        del d['Постриг']
    if 'Дополнительные сведения' in d:
        d['short_biography'] = d['Дополнительные сведения']
        del d['Дополнительные сведения']
    if 'Примечания' in d:
        d['note'] = d['Примечания']
        del d['Примечания']
    if 'Комментарий' in d:
        d['note'] = d['Комментарий']
        del d['Комментарий']
    if "" in d:
        d['note'] = d[""]
        del d[""]

for d in data:
    try:
        d['works'] = d['Сочинения']
        del d['Сочинения']
    except Exception as e:
        del e

for d in data:
    try:
        d['sources'] = d['Литература']
        del d['Литература']
    except Exception as e:
        del e

for d in data:
    try:
        d['archival_sources'] = d['Архивные источники']
        del d['Архивные источники']
    except Exception as e:
        del e

for d in data:
    try:
        d['short_biography'] = d['Другие сведения']
        del d['Другие сведения']
    except Exception as e:
        del e

for d in data:
    try:
        d['biography'] = d['Подробная биография']
        del d['Подробная биография']
    except Exception as e:
        del e

for d in data:
    try:
        tonsure = d['tonsure']
        for i in tonsure:
            try:
                i['tonsure_date'] = i['Дата']
                del i['Дата']
            except Exception as e:
                del e
            try:
                i['tonsure_status'] = i['Описание']
                del i['Описание']
            except Exception as e:
                del e
    except Exception as e:
        del e

# причисление ID
counter = 0
for d in data:
    counter = counter + 1
    counter1 = str(counter)
    d['id'] = 'pravoslavnoe_duhovenstvo_' + counter1

# записываем обновленный json
with open('test.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
